In [15]:
import numpy as np
A = np.arange(36)
np.random.shuffle(A)
A = A.reshape(6,6)

In [16]:
from numpy.lib.stride_tricks import as_strided

s, k = 2, 2

out_height = (A.shape[0] - k)//s + 1
out_width = (A.shape[1] - k)//s + 1

strided = as_strided(A, shape=(out_height, out_width, k, k), strides=(s*A.strides[0], s*A.strides[1], A.strides[0], A.strides[1]))

In [17]:
strided

array([[[[33, 11],
         [27, 15]],

        [[ 6, 14],
         [32,  5]],

        [[ 1,  7],
         [10, 18]]],


       [[[28, 30],
         [22, 25]],

        [[17,  4],
         [20, 35]],

        [[19, 21],
         [26, 34]]],


       [[[ 2, 16],
         [23,  8]],

        [[29,  0],
         [12,  9]],

        [[31, 13],
         [24,  3]]]])

In [18]:
np.__version__

'1.26.1'

In [19]:
merged_shape = (*strided.shape[:-2], -1)  # Merge the last two dimensions
reshaped_strided = strided.reshape(merged_shape)
max_indices = np.argmax(reshaped_strided, axis=-1)

max_indices

array([[0, 2, 3],
       [1, 3, 3],
       [2, 0, 0]])

In [22]:
# Optionally, unravel the indices
unraveled_indices = np.unravel_index(max_indices, (k, k))

print("Max indices:\n", max_indices)
print("Unraveled indices:\n", unraveled_indices)

Max indices:
 [[0 2 3]
 [1 3 3]
 [2 0 0]]
Unraveled indices:
 (array([[0, 1, 1],
       [0, 1, 1],
       [1, 0, 0]]), array([[0, 0, 1],
       [1, 1, 1],
       [0, 0, 0]]))


In [23]:
# Initialize the shape of the original array and the stride
original_shape = (6, 6)
stride = 2

# Calculate the original indices
original_row_indices = np.arange(0, max_indices.shape[0] * stride, stride)[:, np.newaxis] + unraveled_indices[0]
original_col_indices = np.arange(0, max_indices.shape[1] * stride, stride)[np.newaxis, :] + unraveled_indices[1]

# Pair them together
original_indices = np.stack((original_row_indices, original_col_indices), axis=-1)

In [24]:
original_indices

array([[[0, 0],
        [1, 2],
        [1, 5]],

       [[2, 1],
        [3, 3],
        [3, 5]],

       [[5, 0],
        [4, 2],
        [4, 4]]])

In [25]:
A

array([[33, 11,  6, 14,  1,  7],
       [27, 15, 32,  5, 10, 18],
       [28, 30, 17,  4, 19, 21],
       [22, 25, 20, 35, 26, 34],
       [ 2, 16, 29,  0, 31, 13],
       [23,  8, 12,  9, 24,  3]])

In [31]:
A = np.array([[1,  2,  3,  4],
              [5,  6,  7,  8],
              [9,  10, 11, 12],
              [13, 14, 15, 16]])

A = A.reshape(2, 2, 2, 2)
A

array([[[[ 1,  2],
         [ 3,  4]],

        [[ 5,  6],
         [ 7,  8]]],


       [[[ 9, 10],
         [11, 12]],

        [[13, 14],
         [15, 16]]]])

In [32]:
A = A.swapaxes(-2, -3) ;  A

array([[[[ 1,  2],
         [ 5,  6]],

        [[ 3,  4],
         [ 7,  8]]],


       [[[ 9, 10],
         [13, 14]],

        [[11, 12],
         [15, 16]]]])

In [33]:
A.shape

(2, 2, 2, 2)

In [34]:
np.argmax(A, axis=-1)

array([[[1, 1],
        [1, 1]],

       [[1, 1],
        [1, 1]]])

# FINAL

In [37]:
# Further revised transform function to output a 2D array where each row is a flattened kxk block
def final_transform(matrix, k):
    """
    Transforms a matrix so that each kxk block is flattened and each such flattened block becomes a row in a 2D array.
    
    Parameters:
        matrix (np.array): The matrix to transform.
        k (int): The size of the square blocks.
    
    Returns:
        np.array: The transformed 2D array where each row is a flattened kxk block.
    """
    # Reshape into blocks of size kxk
    reshaped_matrix = matrix.reshape(matrix.shape[0] // k, k, matrix.shape[1] // k, k)
    
    # Swap the last two axes
    swapped_matrix = reshaped_matrix.swapaxes(1, 2)
    
    # Flatten each kxk block and make each such flattened block a row in the output 2D array
    return swapped_matrix.reshape(-1, k * k)

# Testing the final transform function with arange(16)
test_matrix = np.arange(16).reshape(4, 4)
final_transformed_test_matrix = final_transform(test_matrix, 2)
final_transformed_test_matrix




array([[ 0,  1,  4,  5],
       [ 2,  3,  6,  7],
       [ 8,  9, 12, 13],
       [10, 11, 14, 15]])

In [38]:
# Using the final_transform function in the main code

# Step 1: Define the original 4x4 matrix A
A = np.array([[12, 15, 5, 0],
              [3, 11, 3, 7],
              [9, 3, 5, 2],
              [4, 7, 6, 8]])

# Step 2: Transform A using the final_transform function
transformed_A = final_transform(A, 2)

# Step 3: Find max_indices using np.argmax
max_indices = np.argmax(transformed_A, axis=1)

# Step 4: Create a lookup matrix and transform it
lookup = np.arange(16).reshape(4, 4)
transformed_lookup = final_transform(lookup, 2)

# Step 5: Retrieve original indices using max_indices
final_original_indices = transformed_lookup[np.arange(max_indices.size), max_indices]

final_original_indices


array([ 1,  7,  8, 15])

In [39]:
# Implementing the MaxPool class
class MaxPool:
    def __init__(self, k):
        self.k = k
        self.lookup = None

    def forward(self, A):
        # Determine padding and pad A
        pad_height = (self.k - A.shape[0] % self.k) % self.k
        pad_width = (self.k - A.shape[1] % self.k) % self.k
        padded_A = np.pad(A, ((0, pad_height), (0, pad_width)), mode='constant')
        
        # Create lookup matrix if it's None or the shape doesn't match
        if self.lookup is None or self.lookup.shape != padded_A.shape:
            rows, cols = padded_A.shape
            self.lookup = np.arange(rows * cols).reshape(rows, cols)
        
        # Transform A and lookup matrix using the final_transform function
        transformed_A = final_transform(padded_A, self.k)
        transformed_lookup = final_transform(self.lookup, self.k)
        
        # Find max_indices using np.argmax
        max_indices = np.argmax(transformed_A, axis=1)
        
        # Retrieve original indices using max_indices
        final_original_indices = transformed_lookup[np.arange(max_indices.size), max_indices]
        
        # Retrieve the max values and reshape to a 2D array
        max_values = padded_A.flatten()[final_original_indices]
        out_height = padded_A.shape[0] // self.k
        out_width = padded_A.shape[1] // self.k
        return max_values.reshape(out_height, out_width)

# Testing the MaxPool class
maxpool = MaxPool(2)
A = np.array([[12, 15, 5, 0],
              [3, 11, 3, 7],
              [9, 3, 5, 2],
              [4, 7, 6, 8]])
maxpool_output = maxpool.forward(A)
maxpool_output


array([[15,  7],
       [ 9,  8]])

In [40]:
# Modifying the MaxPool class to use cleaner keys for lookups
class MaxPool:
    def __init__(self, k):
        self.k = k
        self.lookups = {}  # Dictionary to store lookups for different sizes

    def forward(self, A):
        # Determine padding and pad A
        pad_height = (self.k - A.shape[0] % self.k) % self.k
        pad_width = (self.k - A.shape[1] % self.k) % self.k
        padded_A = np.pad(A, ((0, pad_height), (0, pad_width)), mode='constant')
        
        # Create or retrieve lookup matrix using a cleaner key
        key = f"{padded_A.shape[0]}x{padded_A.shape[1]}"
        if key not in self.lookups:
            rows, cols = padded_A.shape
            self.lookups[key] = np.arange(rows * cols).reshape(rows, cols)
        
        lookup = self.lookups[key]
        
        # Transform A and lookup matrix using the final_transform function
        transformed_A = final_transform(padded_A, self.k)
        transformed_lookup = final_transform(lookup, self.k)
        
        # Find max_indices using np.argmax
        max_indices = np.argmax(transformed_A, axis=1)
        
        # Retrieve original indices using max_indices
        final_original_indices = transformed_lookup[np.arange(max_indices.size), max_indices]
        
        # Retrieve the max values and reshape to a 2D array
        max_values = padded_A.flatten()[final_original_indices]
        out_height = padded_A.shape[0] // self.k
        out_width = padded_A.shape[1] // self.k
        return max_values.reshape(out_height, out_width)

# Quick test to ensure functionality remains the same
maxpool = MaxPool(2)
A1 = np.array([[12, 15, 5, 0],
               [3, 11, 3, 7],
               [9, 3, 5, 2],
               [4, 7, 6, 8]])

output1 = maxpool.forward(A1)
output1


array([[15,  7],
       [ 9,  8]])

In [42]:
def test_maxpool_basic():
    # Initialize MaxPool with k=2
    maxpool = MaxPool(2)
    
    # Define a 4x4 matrix
    A = np.array([[12, 15, 5, 0],
                  [3, 11, 3, 7],
                  [9, 3, 5, 2],
                  [4, 7, 6, 8]])
    
    # Expected output based on manual calculation
    expected_output = np.array([[15, 7],
                                [9, 8]])
    
    # Run the forward method
    output = maxpool.forward(A)
    
    # Check if the output matches the expected output
    assert np.array_equal(output, expected_output), f"Expected {expected_output}, got {output}"

# Run the first test
test_maxpool_basic()

In [45]:
from time import time

# Flushing the cache by reinitializing the MaxPool instance
maxpool = MaxPool(2)

# Timing the first run of the test after flushing the cache
start_time = time()
test_maxpool_basic()
first_run_time_flushed = time() - start_time

# Timing the second run of the test after flushing the cache
start_time = time()
test_maxpool_basic()
second_run_time_flushed = time() - start_time

first_run_time_flushed, second_run_time_flushed


(0.0003509521484375, 0.0005297660827636719)

In [48]:
# Implementing the second test: Padding Test

def test_maxpool_padding():
    # Initialize MaxPool with k=2
    maxpool = MaxPool(2)
    
    # Define a 5x5 matrix
    A = np.array([[12, 15, 5, 0, 1],
                  [3, 11, 3, 7, 2],
                  [9, 3, 5, 2, 3],
                  [4, 7, 6, 8, 4],
                  [1, 2, 3, 4, 5]])
    
    # Expected output based on manual calculation
    expected_output = np.array([[15, 7, 2],
                                [9, 8, 4],
                                [2, 4, 5]])
    
    # Run the forward method
    output = maxpool.forward(A)
    
    # Check if the output matches the expected output
    assert np.array_equal(output, expected_output), f"Expected {expected_output}, got {output}"

# Run the second test
test_maxpool_padding()


In [49]:
# Implementing the third test: Different k Test

def test_maxpool_diff_k():
    # Initialize MaxPool with k=3
    maxpool = MaxPool(3)
    
    # Define a 6x6 matrix
    A = np.array([[12, 15, 5, 0, 1, 2],
                  [3, 11, 3, 7, 2, 4],
                  [9, 3, 5, 2, 3, 1],
                  [4, 7, 6, 8, 4, 6],
                  [1, 2, 3, 4, 5, 2],
                  [2, 3, 5, 1, 0, 3]])
    
    # Expected output based on manual calculation
    expected_output = np.array([[15, 7],
                                [7, 8]])
    
    # Run the forward method
    output = maxpool.forward(A)
    
    # Check if the output matches the expected output
    assert np.array_equal(output, expected_output), f"Expected {expected_output}, got {output}"

# Run the third test
test_maxpool_diff_k()


In [50]:
# Adjusting the fourth test: Reuse Lookup Test

def test_maxpool_reuse_lookup():
    # Initialize MaxPool with k=2
    maxpool = MaxPool(2)
    
    # Define a 4x4 matrix for the first run
    A1 = np.array([
        [12, 15, 5, 0],
        [3, 11, 3, 7],
        [9, 3, 5, 2],
        [4, 7, 6, 8]
    ])
    
    # Define a 5x5 matrix for the second run
    A2 = np.array([
        [12, 15, 5, 0, 1],
        [3, 11, 3, 7, 2],
        [9, 3, 5, 2, 3],
        [4, 7, 6, 8, 4],
        [1, 2, 3, 4, 5]
    ])
    
    # Corrected expected output for A1 based on manual calculation
    expected_output1 = np.array([
        [15, 7],
        [9, 8]
    ])
    
    # Corrected expected output for A2 based on manual calculation
    expected_output2 = np.array([
        [15, 7, 2],
        [9, 8, 4],
        [2, 4, 5]
    ])
    
    # Run the forward method for A1 and check output
    output1 = maxpool.forward(A1)
    assert np.array_equal(output1, expected_output1), f"Expected {expected_output1}, got {output1}"
    
    # Run the forward method for A2 and check output
    output2 = maxpool.forward(A2)
    assert np.array_equal(output2, expected_output2), f"Expected {expected_output2}, got {output2}"
    
    # Run the forward method for A1 again and check output
    output1_again = maxpool.forward(A1)
    assert np.array_equal(output1_again, expected_output1), f"Expected {expected_output1}, got {output1_again}"

# Run the fourth test
test_maxpool_reuse_lookup()
